In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
#from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:


# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'D:\\Users\\kmahendra\\kaggle_Kushwanth_Malariacells_VGG19\\cell_images\\Train'
valid_path = 'D:\\Users\\kmahendra\\kaggle_Kushwanth_Malariacells_VGG19\\cell_images\\Test'

# add preprocessing layer to the front of VGG
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  
  
  # useful for getting number of classes
folders = glob('D:\\Users\\kmahendra\\kaggle_Kushwanth_Malariacells_VGG19\\cell_images\\Train\\*')
  

In [3]:
folders

['D:\\Users\\kmahendra\\kaggle_Kushwanth_Malariacells_VGG19\\cell_images\\Train\\Parasitized',
 'D:\\Users\\kmahendra\\kaggle_Kushwanth_Malariacells_VGG19\\cell_images\\Train\\Uninfected']

In [4]:
#pwd()
#D:\\Users\\kmahendra\\kaggle_Kushwanth_Malariacells_VGG19\\cell_images\\Train\\

In [5]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('D:\\Users\\kmahendra\\kaggle_Kushwanth_Malariacells_VGG19\\cell_images\\Train',
                                                 target_size = (224, 224),
                                                 batch_size = 1,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('D:\\Users\\kmahendra\\kaggle_Kushwanth_Malariacells_VGG19\\cell_images\\Test',
                                            target_size = (224, 224),
                                            batch_size = 1,
                                            class_mode = 'categorical')



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=2,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/2
 2778/21518 [==>...........................] - ETA: 2:10:25 - loss: 1.4714 - accuracy: 0.7203

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

import tensorflow as tf

from keras.models import load_model

model.save('modelmalaria_vgg19.h5')
